<a href="https://colab.research.google.com/github/turing-usp/RL_Truco/blob/main/Truco_Deep_Sarsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ambiente

In [1]:
import gym
from gym import spaces
import numpy as np
import random

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Dicionário do deck (apenas usado pra visualização se quiser e não internamente)
dict_deck = {
    14: '4p',  # 4 de paus (Zap)
    13: '7c',  # 7 de copas (Copeta)
    12: 'Ae',  # Ás de espadas (Espadilha)
    11: '7o',  # 7 de ouros (Ourito)
    10: '3',
    9: '2',
    8: 'A',
    7: 'K',
    6: 'J',
    5: 'Q',
    4: '7',
    3: '6',
    2: '5',
    1: '4',
    0: 'Carta já jogada'
}

In [12]:
class TrucoMineiroEnv(gym.Env):
  def __init__(self):
    # Cria o deck
    self.deck =  self._create_deck()
    # Contador de mãos jogadas
    self.turn = 0
    # Placar 0=oponente, 1=agente
    self.score = [0,0]
    # Aleatoriza quem começa (0=oponente, 1=agente)
    self.first_player = random.randint(0, 1)
    # Definindo o espaço de ação (0, 1, 2 representam as cartas na mão do agente)
    self.action_space = spaces.Discrete(3)
    # Definindo o espaço de observação (carta jogada pelo oponente, cartas na mão do agente, estado da primeira mão)
    self.observation_space = spaces.Tuple((
      spaces.Discrete(15),  # Carta jogada pelo oponente, 0 se for a vez do agente
      spaces.MultiDiscrete([15]*3),  # Cartas na mão do agente (0 representa carta jogada)
      spaces.Discrete(4)  # Estado da primeira mão (0 - essa é a primeira mão, 1 - oponente ganhou, 2 - empate, 3 - agente ganhou)
    ))
    self.observation_space.n = 15 * 15 * 15 * 15 * 4
    # Variáveis de estado
    self.opponent_card = 0 if self.first_player == 1 else self.draw() # Carta jogada pelo oponente (nenhuma ou aleatório)
    self.agent_cards = np.sort([self.draw(), self.draw(), self.draw()])[::-1]
    # Agente compra 3 cartas
    self.first_hand_winner = 0  # Estado da primeira mão

  def _create_deck(self):
    # Deck de cartas com 4p=14 > 7c=13 > Ae=12 > 7o=11 > 3=10 > 2=9 > A=8 > K=7 > J=6 > Q=5 > 7=4 > 6=3 > 5=2 > 4=1
    # 1 de cada manilha, 3 cartas A, 3 cartas 4, 2 cartas 7 e as 4 das demais
    return 1*[14] + 1*[13] + 1*[12] + 1*[11] + 4*[10] + 4*[9] + 3*[8] + 4*[7] + 4*[6] + 4*[5] + 2*[4] + 4*[3] + 4*[2] + 3*[1]

  def draw(self):
    # Compra uma carta do deck
    if self.deck:
      card_index = random.randint(0, len(self.deck) - 1)
      card = self.deck.pop(card_index)
      return card
    else:
      return None

  def step(self, action):
    # Verifica se a ação é válida (0, 1 ou 2)
    if action not in [0, 1, 2]:
      raise ValueError("Invalid action. Action must be 0, 1, or 2.")

    # Executa a ação (joga uma carta)
    player_card = self.agent_cards[action]
    self.agent_cards[action] = 0  # Marca a carta como jogada

    # Se não tiver ação do oponente, joga uma carta aleatória para ele
    if self.opponent_card == 0:
      self.opponent_card = self.draw()

    # Se alguém não jogou, levanta erros (não é para acontecer)
    if self.opponent_card == 0:
      raise ValueError("Opponent has no card set.")
    if player_card == 0:
      raise ValueError("Player has no card set.")

    # Determina o vencedor da mão
    hand_winner = self._determine_hand_winner(self.opponent_card, player_card)

    # Determina o vencedor da rodada, se existir e atualiza o placar
    round_winner = self._determine_round_winner(hand_winner)
    if round_winner == 1 or round_winner == 3:
      self.score[0 if round_winner == 1 else 1] += 1

    # Determina quem ganhou a primeira mão se estiver nela
    if self.turn == 0:
      self.first_hand_winner = hand_winner

    # Define quem começa jogando a próxima mão (mantém em caso de empate, senão quem ganhou começa a próxima)
    if hand_winner != 2:
      self.first_player = min(hand_winner - 1, 1)

    # Se o oponente começar, ele joga uma carta aleatória, senão ele começa sem carta
    if self.first_player == 0:
      self.opponent_card = self.draw()
    else:
      self.opponent_card = 0

    # Avança o turno
    self.turn += 1

    # Determina a recompensa (0 para empates ou rodada inacabada, +1 vitória, -1 derrota)
    reward = 0 if round_winner == 0 else round_winner - 2

    # Sort na mão do agente
    self.agent_cards = np.sort(self.agent_cards)[::-1]

    # Retorna a observação, a recompensa (-1, 0 ou 1) se a rodada acabou ou 0 se a rodada não acabou e a flag de rodada acabada
    observation = (self.opponent_card, self.agent_cards, self.first_hand_winner)
    if round_winner != 0:
      self.reset(reset_score=False)
    done = False if 12 not in self.score else True
    return {'observation' : observation, 'reward' : reward, 'done' : done}

  def reset(self, seed = None, reset_score = True):
    # Reseta o ambiente
    super().reset(seed=seed)
    self.deck = self._create_deck()
    self.turn = 0
    if reset_score:
      self.score = [0,0]
    self.first_player = random.randint(0, 1)
    self.opponent_card = 0 if self.first_player == 1 else self.draw()
    self.agent_cards = np.sort([self.draw(), self.draw(), self.draw()])[::-1]
    self.first_hand_winner = 0
    return {'oberservation' : (self.opponent_card, self.agent_cards, self.first_hand_winner)}

  def _determine_hand_winner(self, opponent_card, agent_card):
    # Lógica para determinar o vencedor de uma mão (1=oponente ganha 2=empate 3=agente ganha)
    if agent_card > opponent_card:
      return 3
    if agent_card < opponent_card:
      return 1
    return 2

  def _determine_round_winner(self, hand_winner):
    # Lógica para determinar o vencedor de uma rodada (0=indeterminado 1=oponente ganha 2=empate 3=agente ganha)
    if self.turn == 2 or self.first_hand_winner == 2: # Terceiro turno ou primeira mão empatou
      return hand_winner
    if self.first_hand_winner == 1 and hand_winner != 3: # Oponente ganha primeira mão
      return 1
    if self.first_hand_winner == 3 and hand_winner != 1: # Agente ganha primeira mão
      return 3
    return 0 # Default

In [13]:
# Testes no ambiente
truco = TrucoMineiroEnv()
observation = truco.reset()
done = False
total_reward = 0

while not done:
    print(f"Observação (opponent_card, agent_cards[], first_hand_winner): {observation}")
    print(f"Cartas do agente: {[dict_deck[card_value] for card_value in truco.agent_cards]}")
    if dict_deck[truco.opponent_card] == 'Carta indisponível':
        print(f"Carta do oponente: ele joga depois")
    else:
        print(f"Carta do oponente: {dict_deck[truco.opponent_card]}")
    while True:
        action = random.randint(0, 2)  # Escolhe uma ação aleatória (trocar pelo agente)
        if truco.agent_cards[action] != 0:
            break
    print(f"Carta jogada pelo agente: {dict_deck[truco.agent_cards[action]]}")
    result = truco.step(action)
    observation, reward, done = result['observation'], result['reward'], result['done']
    total_reward += reward

    print(f"Recompensa obtida neste passo: {reward}")
    print(f"Recompensa acumulada: {total_reward}")
    print(f"Placar: Agente {truco.score[1]} x {truco.score[0]} Oponente\n")

Observação (opponent_card, agent_cards[], first_hand_winner): {'oberservation': (0, array([13,  5,  2]), 0)}
Cartas do agente: ['7c', 'Q', '5']
Carta do oponente: Carta já jogada
Carta jogada pelo agente: Q
Recompensa obtida neste passo: 0
Recompensa acumulada: 0
Placar: Agente 0 x 0 Oponente

Observação (opponent_card, agent_cards[], first_hand_winner): (9, array([13,  2,  0]), 1)
Cartas do agente: ['7c', '5', 'Carta já jogada']
Carta do oponente: 2
Carta jogada pelo agente: 5
Recompensa obtida neste passo: -1
Recompensa acumulada: -1
Placar: Agente 0 x 1 Oponente

Observação (opponent_card, agent_cards[], first_hand_winner): (8, array([13,  0,  0]), 1)
Cartas do agente: ['K', 'Q', 'Q']
Carta do oponente: Carta já jogada
Carta jogada pelo agente: K
Recompensa obtida neste passo: 0
Recompensa acumulada: -1
Placar: Agente 0 x 1 Oponente

Observação (opponent_card, agent_cards[], first_hand_winner): (0, array([5, 5, 0]), 3)
Cartas do agente: ['Q', 'Q', 'Carta já jogada']
Carta do oponent

## Import the necessary software libraries:

In [5]:
import random
import copy
import gym
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn as nn
from torch.optim import AdamW
from tqdm import tqdm

## Create and prepare the environment

### Create the environment

In [15]:
env = TrucoMineiroEnv()

In [17]:
state_dims = env.observation_space.n
num_actions = env.action_space.n
print(f"MountainCar env: State dimensions: {state_dims}, Number of actions: {num_actions}")

MountainCar env: State dimensions: 202500, Number of actions: 3


### Prepare the environment to work with PyTorch

In [ ]:
class PreprocessEnv(gym.Wrapper):

    def __init__(self, env):
        gym.Wrapper.__init__(self, env)

    def reset(self):
        obs = self.env.reset()
        return torch.from_numpy(obs).unsqueeze(dim=0).float()

    def step(self, action):
        action = action.item()
        next_state, reward, done, info = self.env.step(action)
        next_state = torch.from_numpy(next_state).unsqueeze(dim=0).float()
        reward = torch.tensor(reward).view(1, -1).float()
        done = torch.tensor(done).view(1, -1)
        return next_state, reward, done, info

In [ ]:
env = PreprocessEnv(env)

In [ ]:
state = env.reset()
action = torch.tensor(0)
next_state, reward, done, _ = env.step(action)
print(f"Sample state: {state}")
print(f"Next state: {next_state}, Reward: {reward}, Done: {done}")

## Create the Q-Network and policy

<br><br>

### Create the Q-Network: $\hat q(s,a| \theta)$

In [ ]:
q_network = nn.Sequential(
    nn.Linear(state_dims, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, num_actions))

### Create the target Q-Network: $\hat q(s, a|\theta_{targ})$

In [ ]:
target_q_network = copy.deepcopy(q_network).eval()

### Create the $\epsilon$-greedy policy: $\pi(s)$

In [ ]:
def policy(state, epsilon=0.):
    if torch.rand(1) < epsilon:
        return torch.randint(num_actions, (1, 1))
    else:
        av = q_network(state).detach()
        return torch.argmax(av, dim=-1, keepdim=True)

### Plot the cost to go: $ - \max_a \hat q(s,a|\theta)$

In [ ]:
plot_cost_to_go(env, q_network, xlabel='Car Position', ylabel='Velocity')

## Create the Experience Replay buffer

<br>
<div style="text-align:center">
    <p>A simple buffer that stores transitions of arbitrary values, adapted from
    <a href="https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html#training">this source.</a></p>
</div>


In [ ]:
class ReplayMemory:

    def __init__(self, capacity=1000000):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def insert(self, transition):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = transition
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        assert self.can_sample(batch_size)

        batch = random.sample(self.memory, batch_size)
        batch = zip(*batch)
        return [torch.cat(items) for items in batch]

    def can_sample(self, batch_size):
        return len(self.memory) >= batch_size * 10

    def __len__(self):
        return len(self.memory)

## Implement the algorithm

</br></br>

In [ ]:
def deep_sarsa(q_network, policy, episodes, alpha=0.001,
               batch_size=32, gamma=0.99, epsilon=0.05):
    optim = AdamW(q_network.parameters(), lr=alpha)
    memory = ReplayMemory()
    stats = {'MSE Loss': [], 'Returns': []}

    for episode in tqdm(range(1, episodes + 1)):
        state = env.reset()
        done = False
        ep_return = 0
        while not done:
            action = policy(state, epsilon)
            next_state, reward, done, _ = env.step(action)
            memory.insert([state, action, reward, done, next_state])

            if memory.can_sample(batch_size):
                state_b, action_b, reward_b, done_b, next_state_b = memory.sample(batch_size)
                qsa_b = q_network(state_b).gather(1, action_b)
                next_action_b = policy(next_state_b, epsilon)
                next_qsa_b = target_q_network(next_state_b).gather(1, next_action_b)
                target_b = reward_b + ~done_b * gamma * next_qsa_b
                loss = F.mse_loss(qsa_b, target_b)
                q_network.zero_grad()
                loss.backward()
                optim.step()

                stats['MSE Loss'].append(loss.item())

            state = next_state
            ep_return += reward.item()

        stats['Returns'].append(ep_return)

        if episode % 10 == 0:
            target_q_network.load_state_dict(q_network.state_dict())

    return stats

In [ ]:
stats = deep_sarsa(q_network, policy, 2500, epsilon=0.01)

## Show results

### Plot execution stats

In [ ]:
plot_stats(stats)

### Plot the cost to go: $ - \max_a \hat q(s,a|\theta)$

In [ ]:
plot_cost_to_go(env, q_network, xlabel='Car Position', ylabel='Velocity')

### Show resulting policy: $\pi(s)$

In [ ]:
plot_max_q(env, q_network, xlabel='Car Position', ylabel='Velocity',
           action_labels=['Back', 'Do nothing', 'Forward'])

### Test the resulting agent

In [ ]:
test_agent(env, policy, episodes=2)

## Resources

[[1] Deep Reinforcement Learning with Experience Replay Based on SARSA](https://www.researchgate.net/publication/313803199_Deep_reinforcement_learning_with_experience_replay_based_on_SARSA)